In [ ]:
file_id = '1NuLCJDkX5zgpGUl2hTgPdA99y3akeH4C'
fileLink = 'https://drive.google.com/uc?id=' + file_id

In [ ]:
import pandas as pd
df = pd.read_csv(fileLink)
print(df)

     is female  race/ethnicity  parental level of education  free lunch  \
0            0               3                            3           0   
1            1               3                            3           0   
2            1               4                            3           0   
3            1               4                            3           1   
4            1               3                            3           0   
..         ...             ...                          ...         ...   
995          0               2                            0           0   
996          0               3                            0           0   
997          0               3                            0           0   
998          1               2                            0           0   
999          0               3                            0           0   

     test preparation course  math exam  reading exam  writing exam  
0                          0 

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

prediction_data = df[['is female', 'race/ethnicity', 'parental level of education', 'free lunch', 'test preparation course']];
class_data = df['math exam'];

X_train, X_test, y_train, y_test = train_test_split(prediction_data, class_data, test_size=0.2)
model = LogisticRegression()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print('Accuracy:', accuracy)

Accuracy: 0.69
